#### Week 2. MNIST Autoencoding with PytorchLightning + Tensorboard

`Pytorch Lightning`은 nn.module로 대표되는 pytorch 딥러닝 프레임워크 구축 과정을 보조하는, 딥러닝 모델 개발을 쉽고 유연하게 만들어주는 보조 라이브러리입니다. Lightning의 주요 특징은 다음과 같습니다.

1. **Simplified Training Loop** : 기존 `nn.module` 형태에서는, 모델을 정의한 다음에 학습을 시키는 과정에서 loss function을 정의해야 해 학습코드 구축에 어려움이 있었다면, Lightning은 Pytorch class 내에서 training, test, validation step 구축이 가능해 코드를 간소화하고 error를 줄일 수 있습니다.

2. **Code Reusability** : 마찬가지로, Lightning을 사용하면 class 하나를 정의하여 training과 evaluation을 진행할 수 있으므로, 기존 `nn.module`보다 재사용하기 용이합니다.

3. **Hardware Optimization** Lightning은 병렬 및 분산 학습, GPU에 대한 최적화, parameter와 arguments 제시 등 모델을 최적화하는 데 있어 사용하기 용이한 인터페이스를 제공합니다. Lightning의 arguments를 조정함으로, 딥러닝 프레임워크를 task에 알맞은 방식으로 수정할 수 있습니다.

4. **배포의 용이성** Lightning으로 학습한 모델을 Torchscript (JIT) 혹은 ONNX으로 변환할 수 있습니다. 이를 통해 모바일 앱, embedded system, 그리고 클라우드 환경에서 딥러닝 모델을 사용할 수 있습니다.

이번 주차 학습 목표는 다음과 같습니다.

1. 기존 CNN 구조로 학습한 MNIST Autoencoder 딥러닝 프레임워크를 Pytorch Lightning class으로 구축하기

2. Pytorch Lightning의 여러 parameter와 arguments를 확인하고, 이를 사용해 Lightning class를 최적화하기

3. Lightning 구조의 forward, step, 그리고 configure_optimizer에 대해 이해하기

4. 모델 학습의 실시간 현황을 파악할 수 있는 `tensorboard`을 이해하고, Autoencoder로 학습한 reconstructed image를 original image와 비교하기

PytorchLightning Github link : https://github.com/Lightning-AI/lightning


In [ ]:
from IPython.display import clear_output

!pip install pytorch-lightning

clear_output()

In [ ]:
## 필요 라이브러리 import

import pytorch_lightning as pl
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from pytorch_lightning.loggers import TensorBoardLogger
import os

# MNIST 데이터 받아오기
transform=transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST('./', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./', train=False, download=True, transform=transform)

## Tensorboard logger 정의
logger = TensorBoardLogger(save_dir='logs/')

100%|██████████| 9912422/9912422 [00:00<00:00, 155094477.29it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 117379548.28it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 50864484.74it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 4391546.51it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
for index, _ in enumerate(datasets.__all__):
  if (index + 1) % 5 == 0:
    print('')
  print(f'#{index+1}: {_}', end = ' ')

#1: LSUN #2: LSUNClass #3: ImageFolder #4: DatasetFolder 
#5: FakeData #6: CocoCaptions #7: CocoDetection #8: CIFAR10 #9: CIFAR100 
#10: EMNIST #11: FashionMNIST #12: QMNIST #13: MNIST #14: KMNIST 
#15: StanfordCars #16: STL10 #17: SUN397 #18: SVHN #19: PhotoTour 
#20: SEMEION #21: Omniglot #22: SBU #23: Flickr8k #24: Flickr30k 
#25: Flowers102 #26: VOCSegmentation #27: VOCDetection #28: Cityscapes #29: ImageNet 
#30: Caltech101 #31: Caltech256 #32: CelebA #33: WIDERFace #34: SBDataset 
#35: VisionDataset #36: USPS #37: Kinetics #38: HMDB51 #39: UCF101 
#40: Places365 #41: Kitti #42: INaturalist #43: LFWPeople #44: LFWPairs 
#45: KittiFlow #46: Sintel #47: FlyingChairs #48: FlyingThings3D #49: HD1K 
#50: Food101 #51: DTD #52: FER2013 #53: GTSRB #54: CLEVRClassification 
#55: OxfordIIITPet #56: PCAM #57: Country211 #58: FGVCAircraft #59: EuroSAT 
#60: RenderedSST2 #61: Kitti2012Stereo #62: Kitti2015Stereo #63: CarlaStereo #64: Middlebury2014Stereo 
#65: CREStereo #66: FallingThingsStere

In [ ]:
## Train & Test 데이터 shape 확인
## Train = 60_000개
## Test = 10_000개

print(f'# of train dataset : {len(train_dataset)}')
print(f'# of test dataset : {len(test_dataset)}')

# of train dataset : 60000
# of test dataset : 10000


In [ ]:
## Validation 데이터 생성
## 0.8의 비율로 torch tensor를 random_split 라이브러리로 train_test_split

from torch.utils.data import random_split

total_length = len(train_dataset)

train_ratio = 0.8
train_length = int(total_length * train_ratio)
val_length = total_length - train_length

train_dataset, val_dataset = random_split(train_dataset, [train_length, val_length])

In [ ]:
## Train & Validation 데이터 shape 확인
## Train = 48_000개
## Test = 12_000개

print(f'# of train dataset : {len(train_dataset)}')
print(f'# of val dataset : {len(val_dataset)}')

# of train dataset : 48000
# of val dataset : 12000


In [ ]:
## Batch_size 정의
## 딥러닝 프레임워크는 batch_size에 따라 학습을 진행한 후
## Batch만큼 학습한 이후 loss function을 update합니다.
## ex) batch_size = 128, 128개의 데이터 학습 후 loss function update
## ex) batch_size = 32, 32개의 데이터 학습 후 loss function update
## batch_size가 높을 수록 loss function update를 적게 하기에 처리 속도가 빠른 대신, 각각의 데이터의 특성을 덜 반영함
## 일반적으로 32 ~ 128의 batch_size를 권장하고 있습니다.

TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 128
VALID_BATCH_SIZE = 128

In [ ]:
## Pytorch 프레임워크는 DataFrame형태가 아닌 DataLoader 형태를 입력값으로 받습니다.
## Dataloader 형태로 변환하면서 params 형태로 여러 arguments을 추가로 넣을 수 있습니다.
## {'batch_size' : 배치 사이즈
##  'shuffle' : 데이터 셔플 (train에 추천, test, val에는 비추천)
##  'num_workers' : 딥러닝 학습을 보조하는 worker의 수 정의 (0~8)
##  'pin_memory' : CPU -> GPU 처리속도 향상}

train_params = ####

test_params = ####

val_params = ####

train_loader = ####
test_loader = ####
val_loader = ####

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
## AutoEncoder with PytorchLightning

from torch.optim import AdamW, Optimizer
import torch.nn.functional as F
from torchvision.utils import make_grid

class MNISTAutoEncoder(pl.LightningModule):

    ## __init__ 함수
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.save_hyperparameters()

        ## AutoEncoder Encoder & Decoder 구조
        ## __init__ 함수에서 input data가 거칠 layer을 정의합니다.
        self.encoder = ####

        self.decoder = ####

    ## forward 함수
    ## forward 함수는 학습이 끝난 후 test을 진행할 때,
    ## input data가 거치는 과정을 나타낸 함수입니다. bc) 추가학습이 필요없기 때문에!
    def forward(self, x):
        ####

    ## training 함수
    ## 학습과정 & loss function update을 다룹니다.
    ## x = data
    ## x -> encoder -> decoder -> x_hat -> predicted label
    ## x_hat과 x의 차이를 mse_loss으로 계산해 loss function을 업데이트하는 방식으로 학습을 진행합니다.
    def training_step(self, batch, batch_idx):
        ####

    ## test 함수
    def test_step(self, batch, batch_idx):
        ####

    ## validation 함수
    def validation_step(self, batch, batch_idx):
        ####

    ## Tensorboard 이미지 표시용 함수
    ## epoch 학습을 마친 후 하나의 batch에 대해
    ## 원본 배치의 이미지를 형상화한 결과와
    ## 학습중인 배치의 이미지를 형상화한 결과를
    ## logger (tensorboard) 에 결과값을 보냅니다.
    ## Tensorboard에서 각 epoch에 대한 학습 이미지의 변화를 확인할 수 있습니다.
    def on_test_epoch_end(self):
        x, y = next(iter(self.test_dataloader()))
        x = x.to(self.device)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        original_grid = make_grid(x, nrow=8).cpu()
        reconstructed_grid = make_grid(x_hat, nrow=8).cpu()
        self.logger.experiment.add_image("original_images_test", original_grid, self.current_epoch)
        self.logger.experiment.add_image("reconstructed_images_test", reconstructed_grid, self.current_epoch)

    ## Config 함수
    ## optimizer으로 AdamW 함수를 사용하고,
    ## config을 따로 customize하기 위해 **self.config.optim.optimizer 코드를 사용합니다.
    def configure_optimizers(self):
        ####

    ## Train, Test, Validation 데이터 받아오기
    def train_dataloader(self):
        return train_loader

    def test_dataloader(self):
        return test_loader

    def val_dataloader(self):
        return val_loader

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

## checkpoint 정의
## "self.log"으로 정의한 어느 지표를 기준으로 Best Model을 저장할지 정의합니다
## 보통 "val_loss", "min" 혹은 "val_acc", "max" 지표를 사용합니다.
checkpoint = ModelCheckpoint(
    ####
)

## PytorchLightning Trainer 함수를 정의합니다.
## Accelerator = "gpu", devices = "auto"를 사용해, torch.cuda를 정의하지 않고서도 gpu를 사용할 수 있습니다.
## precision = 16을 사용하면 기존 모델의 절반 용량으로 계산할 수 있습니다 (.half()) ==> 용량 최적화!
## log_ever_n_steps = n번의 step마다 정의한 지표 (val_loss, train_loss...) 지표를 업데이트합니다.
## max_epochs = epoch
## logger = Tensorboard 사용
trainer = Trainer(
    ####
)

## Optimizer params 정의
## class 내의 optimizer를 EasyDict를 사용해 class 밖에서 업데이트할 수 있습니다.
## lr = learning rate
## eps = epsilon
from easydict import EasyDict

config = EasyDict({})
config.optim = {}
config.optim.optimizer = {}
config.optim.optimizer.lr = 1e-4
config.optim.optimizer.eps = 1e-6

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [ ]:
## 정의한 Pytorch Lightning 모델로 학습 진행

trainer.fit(MNISTAutoEncoder(config), train_loader, val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 62.0 K
1 | decoder | Sequential | 65.1 K
---------------------------------------
127 K     Trainable params
0         Non-trainable params
127 K     Total params
0.508     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
## Best optimized model이 저장된 위치
## model checkpoint에 val_loss, min을 정의했으므로,
## val_loss가 가장 낮은 모델의 학습구간이 저장된 위치입니다.

####

In [ ]:
## Best model을 불러와 test data에 대해 evaluation 진행

trainer.test(ckpt_path=checkpoint.best_model_path, dataloaders=[test_loader])

In [ ]:
## Tensorboard 불러오기

%load_ext tensorboard
checkpoint_path = trainer.checkpoint_callback.best_model_path
tensorboard_logdir = os.path.dirname(os.path.dirname(checkpoint_path))
%tensorboard --logdir {tensorboard_logdir} --port 1323

In [ ]:
## Best Model Checkpoint 저장

from google.colab import files

files.download(checkpoint.best_model_path)